In [ ]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2


def reversed4(variable):
    res=''.join(reversed(variable))
    return res

def parse_subject_name(x):
    subject_types = ["лек.", "лаб.", "пр.", "кр."]
    
    subject_name = x
    if any(subject_type in x for subject_type in subject_types):
        parts = x.split(" ")[:-1]
        subject_name = " ".join(parts)
    return subject_name

In [ ]:
CURRENT_DPATH = os.path.dirname(os.path.abspath("__file__"))
PROJECT_ROOT = os.path.abspath(os.path.join(CURRENT_DPATH, os.pardir))
DATA_DPATH = os.path.join(PROJECT_ROOT, "data")
data_version = '2023-02-26'
src_dpath = os.path.join(DATA_DPATH, data_version)

### Разделение занятий по типам

In [ ]:
fipath = os.path.join(src_dpath, "Subjects+Lessons.csv")
sub_lessons_df = pd.read_csv(fipath, index_col=0)

SUBJECTS = sub_lessons_df['SUBJECTS']


laboratory = []
practise = []
lecture = []
kurs = []
cash = ''
i = 0
while i < len(SUBJECTS):
    j = -1
    while SUBJECTS[i][j] !=' ':
        cash += SUBJECTS[i][j]
        j -= 1
    cash = reversed4(cash)
    if cash == 'пр.':
        practise.append(SUBJECTS[i])
    elif cash == 'лек.':
        lecture.append(SUBJECTS[i])
    elif cash == 'лаб.':
        laboratory.append(SUBJECTS[i])
    else:
        kurs.append(SUBJECTS[i])
    cash = ''
    i += 1
print(len(laboratory))
print(len(practise))
print(len(lecture))
print(len(kurs))

### Число недель обучения

In [ ]:
ALL_WEEKS = 18

### Число пар в семестре для всех групп

In [ ]:
ALL_LESSONS_FOR_ALL_GROUPS = 1180 #295+295+259+331 = 1180,на одной лекции сидит 4 группы, считается как 4 

### Количество групп

In [ ]:
filepath = os.path.join(src_dpath, "Subjects+Groups.csv")
prepare_df = pd.read_csv(filepath, index_col=0)
NUMBER_OF_GROUPS = len(prepare_df.columns.values.tolist())-1
NUMBER_OF_GROUPS

### Количество преподавателей

In [ ]:
filepath = os.path.join(src_dpath, "Subjects+Teachers.csv")
prepare_df = pd.read_csv(filepath, index_col=0)
NUMBER_OF_TEACHERS = len(prepare_df.columns.values.tolist())-1
NUMBER_OF_TEACHERS


### Количество аудиторий

In [ ]:
filepath = os.path.join(src_dpath, "Room+Subject_Type.csv")
prepare_df = pd.read_csv(filepath, index_col=0)
NUMBER_OF_ROOMS = len(prepare_df['room'].tolist())
NUMBER_OF_ROOMS

### Количество уникальных дисциплин

In [ ]:
fname = "Subjects+Groups.csv"
fpath = os.path.join(src_dpath, fname)
sub_group_df = pd.read_csv(fpath, index_col=0)
print(f"Количество уникальных дисциплин (с учётом типа занятия): {sub_group_df['subjects'].nunique()}")

subject_names_parsed = sub_group_df["subjects"].apply(parse_subject_name)
print(f"Количество уникальных дисциплин (без учёта типа занятия): {subject_names_parsed.nunique()}")
SUBJECTS_UNIQUE = subject_names_parsed.nunique()

### Общее количество часов для всех групп

In [ ]:
fpath = os.path.join(src_dpath, "Subjects+Lessons.csv")
sub_lessons_df = pd.read_csv(fpath, index_col=0)
total_load_h = 0
i = 0
while i < len(sub_lessons_df['SUBJECTS_LESSONS_ALL'].tolist()):
   total_load_h += sub_lessons_df.iloc[i, 3]
   i += 1
print(total_load_h) # занятия
print(total_load_h*1.5) # часы
#963.0 занятия у преподавателей, 892 занятия у групп

### Распределение лекций, практик, лабораторных (график гистограмма)

In [ ]:
Names = ['лабораторные', 'практики', 'лекции', 'курсовые']
Type = [len(laboratory), len(practise), len(lecture), len(kurs)]

plt.bar(Names, Type)
plt.xlabel('Тип')
plt.ylabel('Количество пар')
plt.grid()
plt.show()

### Распределение часов по лекциям, практикам и лабоработорным

In [ ]:
Hours_Type = [len(laboratory)*1.5, len(practise)*1.5, len(lecture)*1.5, len(kurs)*1.5]

plt.bar(Names, Hours_Type)
plt.xlabel('Тип')
plt.ylabel('Количество часов')
plt.grid()
plt.show() 

### Распределение кол-ва пар на каждую группу за одну неделю

In [ ]:
GROUPS = ['9491', '9492', '9493', '9494']
LESSONS_OF_GROUPS = [19, 19, 15.5, 17.5] 

plt.bar(GROUPS, LESSONS_OF_GROUPS)
plt.xlabel('Группа')
plt.ylabel('Количество пар')
plt.grid()
plt.show() 

### Распределение часов на каждую группу

In [ ]:
HOURS_OF_GROUPS = np.array(LESSONS_OF_GROUPS)*1.5 # 1 пара = 1,5 часа
plt.bar(GROUPS, HOURS_OF_GROUPS)
plt.xlabel('Группа')
plt.ylabel('Количество часов')
plt.grid()
plt.show() 

### Распределение должностей преподавателей (график)

In [ ]:
fpath = os.path.join(src_dpath, "Teachers+Lessons.csv")
TEACHERS = pd.read_csv(fpath, index_col=0)

professors = []
assistant = []
cash = ''
i = 0
while i < len(TEACHERS):
    j = -1
    while TEACHERS.iloc[i, 0][j] !=' ':
        cash += TEACHERS.iloc[i, 0][j]
        j -= 1
    cash = reversed4(cash)
    if cash == 'professor':
        professors.append(TEACHERS.iloc[i, 0])
    elif cash == 'assistant':
        assistant.append(TEACHERS.iloc[i, 0])
    cash = ''
    i += 1

x1 = ['Профессоры', 'ассистенты' ]
x2 = [len(professors), len(assistant)]
plt.bar(x1, x2)
plt.xlabel('Преподаватели')
plt.ylabel('Количество преподавателей')
plt.grid()
plt.show() 

### Количество подходящих преподавателей (по предметам) на каждую группу (график)

In [ ]:

df1 = pd.read_csv(os.path.join(src_dpath, 'Subjects+Teachers.csv'))
df2 = pd.read_csv(os.path.join(src_dpath, 'Subjects+Groups.csv'))


In [ ]:
subjects = df2['subjects']
 
plt.figure(figsize =(20, 5))
plt.xticks(rotation=90)
plt.bar(subjects, df2['9491'])
plt.ylabel('9491')
plt.show()

plt.figure(figsize =(20, 5))
plt.xticks(rotation=90)
plt.ylabel('9492')
plt.bar(subjects, df2['9492'])
plt.show()

plt.figure(figsize =(20, 5))
plt.xticks(rotation=90)
plt.ylabel('9493')
plt.bar(subjects, df2['9493'])
plt.show()

plt.figure(figsize =(20, 5))
plt.xticks(rotation=90)
plt.ylabel('9494')
plt.bar(subjects, df2['9494'])
plt.show()
    

### Распределение аудиторий (лекционные, практики, лабораторные)

In [ ]:
df3 = pd.read_csv(os.path.join(src_dpath, 'Room+Subject_Type.csv'))
x = np.array(df3)
i = 0
pr = 0
lec = 0
lab = 0
mixed = 0

while i < len(x):
    if x[i][2] == 'пр.':
        pr += 1
    if x[i][2] =='лек.':
        lec += 1
    if x[i][2] =='лаб.':
        lab += 1
    if x[i][2] =='mixed':
        mixed += 1
    i += 1
print(pr)
print(lec)
print(lab)
print(mixed)
data = [pr, lec, lab, mixed]
plt.figure(figsize =(10, 5))
plt.ylabel('количество занятий')
plt.xlabel('тип занятий')
plt.bar(['пр.', 'лек.', 'лаб.', 'mixed'], data)
plt.show()